In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
import pymongo

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw
from climatools.cliradlw.utils import mapband_new2old

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *

import IPython.display as display

In [2]:
output_notebook()

Loading BokehJS ...

In [3]:
def make_query(param=None):
    return {'param.' + name: value for name, value in param.items()}   

In [4]:
def show_html(s): display.display(display.HTML(s))
    
def show_markdown(s): display.display(display.Markdown(s))

In [5]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['i', 'band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [6]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    dims = ['igg', 'g', 'i']
    for dim in dims:
        if dim in ds:
            if ds[dim].shape == (1,): ds = ds.squeeze(dim).drop(dim)
            elif ds[dim].shape == (): ds = ds.drop(dim)
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')     
    return ds['coolrg']

def pltdata_cooling_1():
    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(DS_COOL.sel(i=1)),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(DS_COOL_CRD.sum('g')),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(DS_COOL_WGT.sel(igg=10).sum('g')),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data

def nice_xlims(pltdata=None, prange=None):
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)

def plt_cool(pltdata=None):
    plin = plt_vert_profile_bokeh(pltdata=pltdata,
                                  y_axis_type='linear',
                                  prange=(50, 1050))
    plin.plot_width, plin.plot_height = 400, 500
    plog = plt_vert_profile_bokeh(pltdata=pltdata,
                                  y_axis_type='log',
                                  prange=(.01, 200))
    plog.plot_width, plog.plot_height = 400, 500
    everything = gridplot(plin, plog, ncols=2)
    return everything
    
def script_plt_cooling():
    pltdata = pltdata_cooling_1()
    p = plt_cool(pltdata=pltdata)
    show(p)
    show_markdown('''*FIGURE.* Cooling rate profiles.''')

In [7]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels: ds = ds.isel(pressure=ilevels)

    dims = ['i', 'igg', 'g']
    for dim in dims:
        if dim in ds:
            if ds[dim].shape == (1,): ds = ds.squeeze(dim).drop(dim)
            elif ds[dim].shape == (): ds = ds.drop(dim)
    
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df

def tbdata_flux():
    tropopauses = {'mls': 40, 'saw': 45, 'trp': 37}
    atmpro = PARAM['atmpro']
    ilevels = [0, tropopauses[atmpro], -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(DS_FLUX_CRD.sum('g'), ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10',
            'df': fmt_flux(DS_FLUX_WGT.sum('g').sel(igg=10), 
                           ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(DS_FLUX.sel(i=1), ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data

def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        show_markdown(f'''*TABLE.*  Fluxes. {d['label']}''')
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        show_markdown(
            f'''*TABLE.*  Fluxes.  ({d['label']}) - ({d0['label']})''')
        
    show_tb(benchmark)
    for d in others: show_tb(d)
    for d in others: show_tbdiff(d=d, d0=benchmark)
        
def script_tb_flux():
    data = tbdata_flux()
    tb_flux(tbdata=data)

In [8]:
def gasbands():
    bmap = mapband_new2old()
    gases = [gas for gas, _ in PARAM['molecule'].items()] 
    bands = [bmap[b] for b in PARAM['band']]
    return list(itertools.product(gases, bands))

def print_bestfit_params():
    df = pd.DataFrame()
    for gas, band in gasbands():
        param = bestfits.kdist_params(molecule=gas, band=band)
        srs = pd.Series(param)
        df[f'({gas}, {band})'] = srs
    df.fillna('-', inplace=True)
    pd.set_option('display.max_colwidth', -1)
    display.display(df.sort_index())
    pd.reset_option('display.max_colwidth')
        
def print_input_param():
    df_clirad = pd.Series(PARAM, name='clirad run').to_frame()
    df_lblnew = pd.Series(PARAM_LBLNEW, name='lblnew run compared against').to_frame()
    df = pd.merge(df_clirad, df_lblnew, how='outer', 
                  left_index=True, right_index=True)
    df.fillna('-', inplace=True)
    pd.set_option('display.max_colwidth', -1)
    display.display(df)
    pd.reset_option('display.max_colwidth')
        
def print_lblnew_param():
    pd.set_option('display.max_colwidth', -1)
    display.display(pd.Series(PARAM_LBLNEW).to_frame())
    pd.reset_option('display.max_colwidth')

In [9]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD.sum('g'), DS_COOL.sel(i=1)]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))

In [10]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD.sum('g'), DS_FLUX.sel(i=1)]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))

In [11]:
def script_fluxcoolr_excelfile():
    '''
    Save fluxes and cooling rates by layer by g-interval in excel file
    and make it available for download.
    '''
    fname = 'output_byg.xlsx'
    
    writer = pd.ExcelWriter(fname)
    
    for g in DS_FLUX_WGT.coords['g']:
        df = DS_FLUX_WGT.sel(igg=1).sel(g=g).to_dataframe()
        df.to_excel(writer, f'flux g={int(g)}')
        
    for g in DS_COOL_WGT.coords['g']:
        df = DS_COOL_WGT.sel(igg=1).sel(g=g).to_dataframe()
        df.to_excel(writer, f'cool g={int(g)}')    

In [12]:


    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()

In [13]:
%run param.py

client = pymongo.MongoClient('localhost', 27017)

doc = client.lblnew.bestfit_lw.find_one(make_query(param=PARAM_LBLNEW))
DS_FLUX_CRD = load_output_file(io.StringIO(doc['output_fluxg']))
DS_COOL_CRD = load_output_file(io.StringIO(doc['output_coolrg']))
DS_FLUX_WGT = load_output_file(io.StringIO(doc['output_wfluxg']))
DS_COOL_WGT = load_output_file(io.StringIO(doc['output_wcoolrg']))

doc = client.cliradnew.lw.find_one(make_query(param=PARAM))
DS_FLUX = load_output_file(io.StringIO(doc['output_flux']))
DS_COOL = load_output_file(io.StringIO(doc['output_coolr']))

In [14]:
script()
if 'ng_refs' in PARAM_LBLNEW:
    script_fluxcoolr_excelfile()

### CLIRAD-LW

### Table of Contents

# Input Parameters

,clirad run,lblnew run compared against
atmpro,mls,mls
band,[7],5
commitnumber,1013f91,a06b618
conc,-,-
dv,-,0.001
klin,-,2e-20
molecule,{'o3': 'atmpro'},o3
ng_adju,-,"[0, 0]"
ng_refs,-,"[2, 5]"
nv,-,1000


# Best-fit Parameters

...

*TABLE.*  Fluxes.  (CLIRAD) - (CRD)

# Cooling Rate by Layer

CRD           CLIRAD `coolr_bands`          
            layer    coolrg                layer    coolrg
pressure                                                  
0.000312        1 -0.038974                    1 -0.030565
0.000750        2 -0.043361                    2 -0.034021
0.001052        3 -0.051819                    3 -0.040776
0.001476        4 -0.059933                    4 -0.047274
0.002070        5 -0.067716                    5 -0.053564
0.002904        6 -0.075181                    6 -0.059615
0.004074        7 -0.082304                    7 -0.065457
0.005714        8 -0.089080                    8 -0.071021
0.008015        9 -0.095516                    9 -0.076380
0.011243       10 -0.101634                   10 -0.081442
0.015771       11 -0.107581                   11 -0.086275
0.022122       12 -0.113372                   12 -0.090898
0.031031       13 -0.119173                   13 -0.095292
0.043528       14 -0.124927                   14 -0.099395
0.061057       15 -0.126906                   15 -0.100695
0.085645       16 -0.117832                   16 -0.094502
0.120136       17 -0.069908                   17 -0.057906
0.168516       18  0.032348                   18  0.025798
0.236378       19  0.199115                   19  0.169034
0.331549       20  0.442457                   20  0.387067
0.465100       21  0.775433                   21  0.696504
0.652400       22  1.217520                   22  1.113066
0.915100       23  1.732497                   23  1.608382
1.283650       24  1.993833                   24  1.911683
1.800600       25  1.790441                   25  1.780806
2.525700       26  1.436163                   26  1.432931
3.542800       27  1.061659                   27  1.053176
4.969550       28  0.744368                   28  0.752719
6.970850       29  0.522055                   29  0.553500
9.778100       30  0.351804                   30  0.397210
13.715850      31  0.178967                   31  0.220191
19.239350      32  0.010305                   32  0.034848
26.987250      33 -0.108022                   33 -0.100311
37.855300      34 -0.191344                   34 -0.199762
53.100050      35 -0.261042                   35 -0.287259
73.887500      36 -0.282583                   36 -0.323380
97.662500      37 -0.250037                   37 -0.290708
121.437500     38 -0.216483                   38 -0.249850
145.212500     39 -0.201254                   39 -0.226283
168.987500     40 -0.180020                   40 -0.195896
192.762500     41 -0.144146                   41 -0.153080
216.537500     42 -0.114265                   42 -0.119590
240.312500     43 -0.093807                   43 -0.096879
264.087500     44 -0.074862                   44 -0.076311
287.862500     45 -0.059860                   45 -0.060301
311.637500     46 -0.049904                   46 -0.049714
335.412500     47 -0.041454                   47 -0.040792
359.187500     48 -0.033650                   48 -0.032686
382.962500     49 -0.026999                   49 -0.025847
406.737500     50 -0.021379                   50 -0.020112
430.512500     51 -0.016308                   51 -0.014987
454.287500     52 -0.011774                   52 -0.010454
478.062500     53 -0.007761                   53 -0.006469
501.837500     54 -0.004288                   54 -0.003049
525.612500     55 -0.001214                   55 -0.000039
549.387500     56  0.001574                   56  0.002670
573.162500     57  0.004125                   57  0.005135
596.937500     58  0.006484                   58  0.007401
620.712500     59  0.008655                   59  0.009469
644.487500     60  0.010675                   60  0.011379
668.262500     61  0.012563                   61  0.013151
692.037500     62  0.014295                   62  0.014759
715.812500     63  0.015912                   63  0.016256
739.587500     64  0.017443                   64  0.017662
763.362500     65  0.018855                   65  0.018944
78

# Fluxes by Layer

CRD                          CLIRAD                \
                        flug      fldg      fnetg       flug          fldg   
pressure    level                                                            
0.000000    1     -23.045621  0.000000 -23.045621 -22.641230 -5.323260e-08   
0.000624    2     -23.045625  0.000001 -23.045624 -22.641233  9.587277e-07   
0.000876    3     -23.045627  0.000002 -23.045625 -22.641234  1.419485e-06   
0.001229    4     -23.045630  0.000003 -23.045627 -22.641237  2.219329e-06   
0.001723    5     -23.045636  0.000005 -23.045631 -22.641241  3.568376e-06   
0.002417    6     -23.045644  0.000008 -23.045636 -22.641248  5.790852e-06   
0.003391    7     -23.045658  0.000013 -23.045645 -22.641258  9.392145e-06   
0.004757    8     -23.045679  0.000021 -23.045658 -22.641274  1.514643e-05   
0.006672    9     -23.045712  0.000034 -23.045678 -22.641300  2.424460e-05   
0.009359    10    -23.045763  0.000054 -23.045709 -22.641338  3.849415e-05   
0.013128    11    -23.045839  0.000086 -23.045753 -22.641397  6.065588e-05   
0.018415    12    -23.045955  0.000134 -23.045821 -22.641485  9.490113e-05   
0.025830    13    -23.046128  0.000207 -23.045921 -22.641617  1.475098e-04   
0.036232    14    -23.046388  0.000320 -23.046068 -22.641815  2.279637e-04   
0.050823    15    -23.046773  0.000490 -23.046284 -22.642110  3.505593e-04   
0.071291    16    -23.047354  0.000763 -23.046592 -22.642553  5.496102e-04   
0.100000    17    -23.048300  0.001308 -23.046992 -22.643276  9.509135e-04   
0.140271    18    -23.049828  0.002503 -23.047325 -22.644448  1.847245e-03   
0.196760    19    -23.052095  0.004986 -23.047109 -22.646183  3.754362e-03   
0.275997    20    -23.055152  0.009912 -23.045240 -22.648486  7.644918e-03   
0.387100    21    -23.058720  0.019302 -23.039417 -22.651052  1.530479e-02   
0.543100    22    -23.061658  0.036571 -23.025087 -22.652786  2.991179e-02   
0.761700    23    -23.060732  0.067172 -22.993560 -22.650812  5.676252e-02   
1.068500    24    -23.049716  0.119120 -22.930596 -22.639799  1.042085e-01   
1.498800    25    -23.024793  0.195826 -22.828967 -22.616239  1.781019e-01   
2.102400    26    -22.989374  0.288425 -22.700948 -22.582580  2.717847e-01   
2.949000    27    -22.946132  0.389210 -22.556922 -22.540972  3.738939e-01   
4.136600    28    -22.900467  0.492899 -22.407568 -22.496487  4.775851e-01   
5.802500    29    -22.860691  0.600015 -22.260677 -22.457244  5.868971e-01   
8.139200    30    -22.834582  0.718410 -22.116172 -22.430710  7.135871e-01   
11.417000   31    -22.833744  0.854169 -21.979574 -22.428163  8.652843e-01   
16.014700   32    -22.881561  0.999458 -21.882104 -22.473206  1.030263e+00   
22.464000   33    -23.012168  1.137938 -21.874230 -22.600927  1.184608e+00   
31.510500   34    -23.261521  1.271532 -21.989989 -22.851427  1.327602e+00   
44.200100   35    -23.685476  1.407865 -22.277611 -23.290714  1.466581e+00   
62.000000   36    -24.356033  1.528008 -22.828025 -24.016588  1.586699e+00   
85.775000   37    -25.235308  1.611440 -23.623869 -25.011888  1.671164e+00   
109.550000  38    -25.990445  1.662393 -24.328053 -25.883116  1.723582e+00   
133.325000  39    -26.640612  1.702874 -24.937739 -26.628298  1.765035e+00   
157.100000  40    -27.241178  1.736644 -25.504534 -27.300527  1.799913e+00   
180.875000  41    -27.779946  1.768419 -26.011527 -27.884697  1.832319e+00   
204.650000  42    -28.224245  1.806756 -26.417489 -28.353041  1.869496e+00   
228.425000  43    -28.595373  1.856076 -26.739296 -28.736217  1.915833e+00   
252.200000  44    -28.917807  1.914320 -27.003487 -29.063446  1.970193e+00   
275.975000  45    -29.190796  1.976474 -27.214322 -29.336358  2.028165e+00   
299.750000  46    -29.423698  2.040791 -27.382908 -29.566226  2.088190e+00   
323.525000  47    -29.632424  2.108972 -27.523452 -29.769936  2.151876e+00   
347.300000  48    -29.820538  2.180336 -27.640201 -29.951622  2.218667e+00   
371.075000  49    -29.988326  2.253357 -27.7349

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')